<a href="https://colab.research.google.com/github/Yewon9/STT/blob/main/EDA_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!pip install pydub

In [21]:
import pandas as pd
from pydub import AudioSegment
from tqdm import tqdm

# 일반 음성 데이터

In [14]:
general = pd.read_csv('general.csv')

## 데이터 개수 확인

In [15]:
print(general.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37196 entries, 0 to 37195
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Path    37196 non-null  object
 1   Text    37196 non-null  object
dtypes: object(2)
memory usage: 581.3+ KB
None


In [16]:
print(general.describe())

                                                     Path              Text
count                                               37196             37196
unique                                              37196             37196
top     /content/drive/MyDrive/project/data_aihub/4/5e...  어, 청소 니가 대신 해 줘!
freq                                                    1                 1


In [17]:
missing_text = general[general['Text'] == '']
print(f"빈 텍스트 개수: {len(missing_text)}")

빈 텍스트 개수: 0


In [18]:
duplicate_texts = general['Text'].duplicated().sum()
print(f"중복된 텍스트 개수: {duplicate_texts}")

중복된 텍스트 개수: 0


## 음성 파일 길이 분석

In [19]:
def get_audio_length(path):
  audio = AudioSegment.from_wav(path)
  return len(audio) / 1000

In [ ]:
general['Audio_Length'] = [get_audio_length(path) for path in tqdm(general['Path'])]

  0%|          | 47/37196 [00:21<4:39:08,  2.22it/s]